In [1]:
import sys
import numpy as np
import pandas as pd
import lightgbm as lg
from tqdm import tqdm
import pandas as pd
from pathlib import Path

# from importlib import reload


import catboost
from catboost.utils import get_gpu_device_count

sys.path.append("../../scorer/")
# import orderbook as ob
# Чтобы использовать быстрый ордербук раскомментируйте строку:
import orderbook_fast as ob

from my_orderbook import MyOrderBook

# from  solution import get_simple_features_from_orderbook, get_simple_deals_features
from  solution import get_simple_deals_features

SIDE_BID = 0 
SIDE_ASK = 1

catboost_myob = MyOrderBook()

print(f"curdir: {Path.cwd()}")

curdir: /home/sergey/mnt/st1500/Usr/Sergey/TheJob/Challenges/wunder_summer/wunder_challenge/examples/catboost_local


## Собираем датасет для тренировки модели

In [ ]:
# %%time

cat_features = ["action", "type", "side", "is_snapshot"]

def collect_dataset(data_path):
    '''
        Собираем датасет
    '''

    global catboost_myob
    event_player = ob.EventPlayer(data_path)
    orderbook = ob.OrderBook()

    X = []
    Y = []

    last_deal = [None, None]
    for ev in tqdm(event_player.iget_events(),
                    total=len(event_player),
                    desc="collecting dataset"):

        if ev.action == ob.Action.DEAL:
            last_deal[ev.side] = ev
            catboost_myob.set_last_deal(ev)
        elif ev.action == ob.Action.NEW_CHUNK:
            catboost_myob.clear()
            last_deal = [None, None]

        orderbook.apply_event(ev)
        if ev.need_prediction:
            features = catboost_myob.get_features_from_orderbook(orderbook)
            features += catboost_myob.get_simple_deals_features(last_deal
                                                                , orderbook)

            X.append(features)
            Y.append(ev.Y)

    print(f"Dataset collected: len(X) = {len(X)}")
    return np.array(X), np.array(Y)


X_train, Y_train = collect_dataset("../../data/train_small_A.npz")
X_test, Y_test = collect_dataset("../../data/train_small_B.npz")




collecting dataset:  98%|█████████▊| 10317783/10555835 [12:58<00:14, 16058.46it/s]

 3 clear 
           time  cat_action  cat_type  cat_side  price  amount  \
0   14738881244           3         0         1   1200       2   
1   14739559244           3         0         1   1200       1   
2   14740213383           3         0         1   1200       5   
3   14740354072           3         0         0   1201       1   
4   14740939245           3         0         1   1201       1   
5   14741279447           3         0         1   1200       2   
6   14742604486           3         0         1   1200       1   
7   14742747764           3         0         0   1201       1   
8   14746968122           3         0         1   1200       2   
9   14746968382           3         0         1   1200       2   
10  14746968394           3         0         1   1200       9   
11  14746968402           3         0         1   1200       1   
12  14746968414           3         0         1   1200       7   
13  14746968426           3         0         1   1200       4   


## Обучаем модель градиентного бустинга

In [ ]:
def train_classifier(X_train, Y_train, X_test, Y_test):
    '''
        Обучаем классификатор
    '''
    clf = lg.LGBMClassifier(num_leaves=31, n_estimators=1000, learning_rate=0.1)
    clf.fit(X_train, Y_train, eval_set=[(X_test, Y_test)], 
            eval_metric="auc", early_stopping_rounds=20)
    return clf
 
# clf = train_classifier(X_train, Y_train, X_test, Y_test)

In [ ]:
%%time
def train_catboost(X_train, Y_train, X_test, Y_test):

    task_type = "CPU" #  if (get_gpu_device_count() == 0) else "GPU"
    langevin = False if task_type == "GPU" else True
    print(f"catboost training with {task_type}...")
    print("X_train.shape = ", X_train.shape)
    print("get_gpu_device_count() = ", get_gpu_device_count())

    train_pool = catboost.Pool(X_train, Y_train)     #    , cat_features=cat_features)

    grid = {'learning_rate': [0.03, 0.01],
            # 'score_function': ["Cosine", "L2", "NewtonL2"],
            'depth': [4, 5, 6],
            'l2_leaf_reg': [2, 3, 5, 8, 11],
            # 'nan_mode': ["Min", "Max"],
            # 'fold_len_multiplier': np.linspace(1.1, 3, 3),
            # 'bagging_temperature': np.linspace(1, 3000, 3),

    }

    model = catboost.CatBoostClassifier(

        bootstrap_type="Bayesian",
        # scale_pos_weight=en.scale_pos_weight,
        # scale_pos_weight=1,
        loss_function = "Logloss",
        eval_metric="AUC:hints=skip_train~false",
        langevin=langevin,
        custom_metric=["Recall", "Precision", "Accuracy", "F1", "Kappa", "MCC"],
        # use_best_model=True,
        iterations=100,
        od_type="Iter",
        od_wait=30,
        # rsm=0.5,
        # random_seed=100,
        task_type=task_type,
        boosting_type='Ordered',
        logging_level="Verbose",
        train_dir="grid",

    )

    grid_search_result = model.grid_search(
        grid,
        X=train_pool,
        stratified=True,
        cv=3,
        search_by_train_test_split=False,
        plot=False      # PLOT,

    )


    params = model.get_params()
    for param in params.items():
        print(param)

    return grid_search_result, model

grid_search_result, clf = train_catboost(X_train, Y_train, X_test, Y_test)
# clf = train_classifier(X_train, Y_train, X_test, Y_test)

## Тестируем получившийся классификатор

In [ ]:
last_deals = [None, None]

def process_event_and_predict_proba(event, orderbook):
    if event.action == ob.Action.DEAL:
        last_deals[event.side] = event
    elif event.action == ob.Action.NEW_CHUNK:
        last_deals[:] = [None, None]
        
    if not event.need_prediction:
        return None
    
    features = catboost_myob.get_features_from_orderbook(orderbook)
    features += get_simple_deals_features(last_deals, orderbook)    
    proba = clf.predict_proba([features])[0, 1]
    return proba

from scorer import Scorer

scoring = Scorer("../../data/train_small_C.npz")
roc_auc, (true_ys, pred_probas) = scoring.score(process_event_and_predict_proba)

## Сохраним модель, и решение для отправки готово

In [ ]:
# Сохраним нашу модель
# clf.booster_.save_model("wunder.model")
clf.save_model("wunder.model", format="cbm")
print(f"curdir: {Path.cwd()}")
'''Посмотрите код файла solution.py. 
Он использует те же функции что и этот ноутбук, но уже готов к отправке на серверю. 
Попробуйте создать архив с файлами solution.py и wunder.model и отправить их на проверку.'''